In [ ]:
from IPython.core.display import HTML, display
display(HTML('<script>Jupyter.notebook.clear_all_output(); '+
             'Jupyter.notebook.kernel.restart();</script>'))

In [1]:

import numpy as np;
import plotly.graph_objects as go
from numpy.linalg import *
from scipy.signal import periodogram, convolve, remez, freqz, find_peaks, lfilter, welch;
from scipy.signal.windows import chebwin;
import pandas as pds;
from datetime import datetime
from linregs import *;
from plots import *;
from msk import *;
# from dbutils import *;
from equalization import *; 
from jack_record import record;
import os
from datetime import datetime
import subprocess
from itertools import combinations
    

import plotly.io as pio
import profile; 


In [20]:
### File options ###
home="/home/jradema/Documents/sk/oceans/"

dist=150
# tx_amp = 0.08 # 60Vrms
tx_amp = 0.06 # 40Vrms
# tx_amp = 0.015 # 10Vrms
# tx_amp = 0.025 # 20Vrms
volt = 40 #np.round(tx_amp*1000).astype("uint8")
# trial=1
num_trials = 24
trial_start = 0
angles = range(-90,91,5)
nch=7

msk_ord = 0
data_rate = 500
fs = 2e5
nbits = 10000

rx_dir=os.path.join(home,"rx_outputs");
tx_dir=os.path.join(home,"tx_outputs");
expname='River_PAB2_Van_Atta';
now = datetime.now()
datestr=now.strftime("%m-%d-%Y");
datestr="Range_0124_0127_0130"
expset=expname + "_" + datestr;

try:
    os.mkdir(os.path.join(rx_dir,expset));
    print(f"Created directory {expset} in {rx_dir} for experiment set.")
except FileExistsError as error:
    print(f"Directory {expset} already exists")
    
rx_root = f"fixed_vanatta4x2_dr={data_rate}bps_ord={msk_ord}_Vrms={volt}_{dist}m_1m_single_foam_sep_purui_rx";

#rx_filename = f"vanatta4x2_{nch}ch_rx_nostag_006B_006F_006A_006C_x_001A_004A_004B_004D_18,5kfc_0,0deg_0,5kbps_usrp_2,5m_depth_005B_{volt}Vrms_{dist}m_{dist}m_1m_2foam_sep_{trial}.dat";

tx_filename = f"msk_data_dr={data_rate}_ord={msk_ord}_nbits=10k_fs=2e5.dat";
tx_path = os.path.join(tx_dir,tx_filename);

Directory River_PAB2_Van_Atta_Range_0124_0127_0130 already exists


In [21]:
def get_wf( filename, spb, Fc_c, data_rate, msk_ord, Fs, hyd_num, stretch=1):
    Fc_t = -data_rate * (msk_ord*2+3) /4;
    Fs_new = data_rate * spb;
    segment=np.fromfile(filename,dtype=np.float32).reshape([-1, 7])[:, hyd_num]
#     pcs(segment[:,0])
#     print(f'segment has shape {segment.shape}, Fs={Fs}, Fs_new={Fs_new}')
    return mixer_cfo(Fc_c, [Fc_t, -Fc_t], stretch, Fs, segment, Fs_new)

def fm0_bpsk1(source):
    last_sym = 1;
    polarity = -1;
    output = np.zeros((source.size * 2));
    for i in range(source.size):
        if last_sym != source[i]: # xor = 1;
            output[i*2:i*2+2] = -polarity, -polarity
            polarity = -polarity;
        else: # xor = 0
            output[i*2:i*2+2] = -polarity, polarity
        last_sym = source[i];
    return output;

In [25]:
### EQUALIZE vs. VARIABLE (angle, range, etc.) ### 

# Nfft = 2**(np.ceil(np.log2(Fs*10)));
# att = 300; # attenuation of chebwin in dB

# var_list = [22,23,24,25,27,28,29,30,32,34,36,38,40,42,44,47]
# var_list = [40,47,50,53,56,60,63,66,70,78,80,81,83]
# var_list = [81,83,90,100]
# var_list = [10,20,30,40,50,60]
spb=2
source = pn15_seq[:nbits] * 2 - 1;    

fw_len=15
bw_len=4
train_len = 1000;
lam = 0.999
Fc_c = 18500
Fs=192000

max_step = source.size-train_len

preamble_bits = source[:train_len];
template_fm0 = fm0_bpsk1(preamble_bits);
template = fm0_subcar_filt(template_fm0, spb)
exp_bits = source[train_len:]

var_list = [150] #[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]
#snr_arr = np.zeros((len(var_list),))
#ber_arr = np.zeros((len(var_list),))
volt = 40
snr_dis = np.zeros((len(var_list),7))
ber_dis = np.zeros((len(var_list),7))
# spectrums = np.zeros((len(var_list),Nfft.astype("uint32")))

for i in range(0,len(var_list)):   
    dist = var_list[i]
    print(f"Distance: {dist}")
    
    rx_root = f"fixed_vanatta4x2_dr={data_rate}bps_ord={msk_ord}_Vrms={volt}_{dist}m_1m_single_foam_sep_purui_rx";
    snr_hyd = []
    ber_hyd = []
    for hyd in range(1,8):
        snr_h = []
        ber_h = []
        for trial in range(0,num_trials):
            if (dist == 40 or dist == 50) and trial >= 0 and trial < 4 and data_rate == 500:
                continue
                
            if dist == 150 and trial >=4 and trial < 8 and data_rate == 1000:
                continue
            
            rx_filename = f"{rx_root}_{trial}.dat"
            rx_path = os.path.join(rx_dir,expset,rx_filename); 
            hyd_comb = combinations([0,1,2,3,4,5,6],hyd)
            
            if not os.path.isfile(rx_path):
                print(f'Could not find trial {trial}, skipping...')
                continue

            print(f"Trial: {trial}")
            print(f"Num_Hyd: {hyd}")
            for h in hyd_comb:
                print(h)
                dat1 = get_wf(rx_path, spb, Fc_c, data_rate, msk_ord, Fs, h,)

                # sig = passband[:, 6];
                # window_size = np.floor(len(sig)/100);
                # window = chebwin(window_size.astype("uint32"), att);

                # spectrums[i,:] = welch(sig,Fs,window=window,nfft=Nfft);

                # print(f'dat has shape{dat1.shape}')
                rxfilt = fm0_lowpass_filt(dat1, spb);
                # print(f"rxfilt has shape {rxfilt.shape}")
                scale,corr = linreg_mc_od(rxfilt, template);
                offset = fw_len//3
                #offset  =0;

                pos= np.argmax(corr[:]) + (offset - preamble_bits.size) * spb + 1;
                print(f'pos={pos}')
                bpsk_sample = fm0_bb_decimate(rxfilt[pos:, :], spb, True, True);

                try:
                    d_all, e_all, eq = dfe_sfo_main(bpsk_sample, max_step, spb, lam, fw_len, bw_len, preamble_bits, LDLT_RLS);

                    ber = np.sum(exp_bits != d_all) / max_step;
                    bplot = np.cumsum(exp_bits != d_all) ;
                    avg_len = 20
                    mse = 10*np.log10(convolve(np.abs(e_all)**2,np.ones((avg_len,)))/avg_len)
                    snr = d_all.size/np.sum(np.abs(e_all+d_all-exp_bits)**2);

                except:
                    snr = 1
                    ber = 0.5

                #print('snr is %.2f dB' % (10*np.log10(snr)))
                #print('ber is %.2E' % ber)
                if ber < 1e-4:
                    ber = 1e-4
                    
                ber_h.append(ber)
                snr_h.append(snr)
                
        snr_hyd.append(np.median(snr_h))    
        ber_hyd.append(np.median(ber_h))
        
    snr_dis[i,:] = snr_hyd
    ber_dis[i,:] = ber_hyd
    
    snr_dis.astype("float32").tofile(f"matplot/med_snr_multihyd_vs_range_single_foam_{data_rate}bps_{volt}Vrms_4x2va_{datestr}.bin")
    ber_dis.astype("float32").tofile(f"matplot/med_ber_multihyd_vs_range_single_foam_{data_rate}bps_{volt}Vrms_4x2va_{datestr}.bin")
    
    #snr_arr[i] = 10*np.log10(np.median(snrs));
    #ber_arr[i] = np.median(bers) if np.median(bers) >= 1e-4 else 1e-4;

#pcs(ber_arr)
#pcs(snr_arr)
    

Distance: 150
Trial: 0
Num_Hyd: 1
(0,)
Fc is [[[ 18125. -18875.]]]
[-28.561695-315.6919j  607.28973  -74.09782j]
pos=1644
(1,)
Fc is [[[ 18125. -18875.]]]
[1005.6442  -66.351265j -125.93008-592.4844j  ]
pos=1644
(2,)
Fc is [[[ 18125. -18875.]]]
[-648.1977 -330.34775j -761.2528+1094.8538j ]
pos=1644
(3,)
Fc is [[[ 18125. -18875.]]]
[ -96.081505 +575.31177j -405.90234 -1195.3959j ]
pos=1644
(4,)
Fc is [[[ 18125. -18875.]]]
[1044.055  -19.370123j  830.6938-605.0441j  ]
pos=1644
(5,)
Fc is [[[ 18125. -18875.]]]
[-98.1906  +224.77739j 128.92671-1681.6256j ]
pos=1644
(6,)
Fc is [[[ 18125. -18875.]]]
[ 509.6718 +648.13214j -210.37747-255.74628j]
pos=1644
Trial: 1
Num_Hyd: 1
(0,)
Fc is [[[ 18125. -18875.]]]
[ 232.5351  +366.38516j -616.36084+1119.1659j ]
pos=811
(1,)
Fc is [[[ 18125. -18875.]]]
[244.64299+939.6673j 356.59814+535.946j ]
pos=811
(2,)
Fc is [[[ 18125. -18875.]]]
[810.1673 -487.01996j 867.3254+2162.6245j ]
pos=811
(3,)
Fc is [[[ 18125. -18875.]]]
[606.65295+756.8709j 132.437  +915

pos=1644
(0, 2)
Fc is [[[ 18125. -18875.]]]
[  144.7686  -408.46915j   755.67084 -578.9583j   -733.3492  -217.285j
 -1262.4072 +1159.5682j ]
pos=1644
(0, 3)
Fc is [[[ 18125. -18875.]]]
[ 177.00732 -799.2954j  1340.3654  +220.05084j -200.67378 +755.3252j
 -895.379  -1675.0127j ]
pos=1644
(0, 4)
Fc is [[[ 18125. -18875.]]]
[-363.66064 -28.561184j   422.24158  -3.2260518j 1091.2     +25.179482j
  805.2299 -580.72986j  ]
pos=1644
(0, 5)
Fc is [[[ 18125. -18875.]]]
[  -1.7575073 -518.56757j  905.242     +343.61536j
 -164.69446   +315.57922j   -9.193008 -1885.1221j ]
pos=1644
(0, 6)
Fc is [[[ 18125. -18875.]]]
[ -82.00171-172.2622j   516.15955 -53.92537j  469.27084+625.15375j
 -219.9898 -241.6196j ]
pos=1644
(1, 2)
Fc is [[[ 18125. -18875.]]]
[ 931.0877 -187.06425j -243.7872 -557.5746j  -446.52124-538.5585j
 -799.057  +917.3581j ]
pos=1644
(1, 3)
Fc is [[[ 18125. -18875.]]]
[1024.3923   +96.41647j -373.31927 -866.8888j   199.62859 +735.4814j
 -384.61984-1343.7273j ]
pos=1644
(1, 4)
Fc is [[[

[570.3957   -84.637634j 691.0373  +106.223816j 500.85715 +156.81973j
  58.16712-1385.91j    ]
pos=834
(4, 6)
Fc is [[[ 18125. -18875.]]]
[  540.49677  -19.143326j   591.1686  +174.24734j
   -55.840126-479.55994j  -2059.664   -267.4472j  ]
pos=834
(5, 6)
Fc is [[[ 18125. -18875.]]]
[  400.77484 +165.84712j   101.668686-496.74155j  -130.4806  -615.2832j
 -1810.5486  -441.95734j]
pos=834
Trial: 3
Num_Hyd: 2
(0, 1)
Fc is [[[ 18125. -18875.]]]
[ -316.50873 -148.65887j  -304.71365-2213.1904j   1298.133   -122.87155j
 -2200.4993  -329.20813j]
pos=836
(0, 2)
Fc is [[[ 18125. -18875.]]]
[ -541.8093 +346.23938j   357.0174-2050.3303j  -1748.5728-1461.0226j
  -876.2172-1843.0886j ]
pos=836
(0, 3)
Fc is [[[ 18125. -18875.]]]
[-438.72803  -64.77466j -468.99152-2281.2927j  -710.16284 +539.8291j
 1845.0264  +221.30612j]
pos=836
(0, 4)
Fc is [[[ 18125. -18875.]]]
[-340.8575    -7.2802887j -309.43723-1740.3257j
 -858.90826 -311.54504j    541.1341 -1322.8962j   ]
pos=836
(0, 5)
Fc is [[[ 18125. -18875.]]

[  747.6183-1033.5151j   1057.9688 -305.19006j  3015.5527+3770.5942j
 -2417.413  -685.6844j ]
pos=782
(2, 6)
Fc is [[[ 18125. -18875.]]]
[ 1456.5902 +1678.6019j   935.80634 -496.2296j -1501.6614 -7282.3735j
   875.66516 +197.8157j]
pos=782
(3, 4)
Fc is [[[ 18125. -18875.]]]
[-3827.391  +1625.0928j   1557.6888  -563.46234j   601.98303 +896.70935j
 -4198.7007 +6769.199j  ]
pos=782
(3, 5)
Fc is [[[ 18125. -18875.]]]
[-3069.1392 +1297.1597j    371.77972 -306.93887j  1257.7351   +71.78899j
 -3660.8684 -1540.2052j ]
pos=783
(3, 6)
Fc is [[[ 18125. -18875.]]]
[-2151.0479+1504.093j     127.8725 +481.68112j  1147.1505-2528.7593j
  -523.8031 +956.96893j]
pos=782
(4, 5)
Fc is [[[ 18125. -18875.]]]
[ 1648.8724-4238.993j   -5144.5947 +441.33096j  6519.4155+2044.335j
 -5451.3594+2813.1523j ]
pos=783
(4, 6)
Fc is [[[ 18125. -18875.]]]
[-4061.3865 +2017.3695j  -640.87445+3744.2568j  3048.683  -7110.1377j
   571.71045-1691.6943j]
pos=782
(5, 6)
Fc is [[[ 18125. -18875.]]]
[ 4313.715  +3117.8464j  -4020

pos=1604
(1, 3)
Fc is [[[ 18125. -18875.]]]
[   32.9009    -13.355756j   -30.024143  -24.712315j
  4144.589   -2198.8904j   -2737.8547   +144.04509j ]
pos=1604
(1, 4)
Fc is [[[ 18125. -18875.]]]
[   33.857887  -12.536424j   -30.621689  -26.591274j
 -1826.9458   -548.2913j    -913.48645 +4789.275j   ]
pos=1604
(1, 5)
Fc is [[[ 18125. -18875.]]]
[   34.24314   -12.419016j   -29.955053  -26.98113j
   234.86047 -5060.368j    -2470.1062  +5523.104j   ]
pos=1604
(1, 6)
Fc is [[[ 18125. -18875.]]]
[   33.767345  -12.6837225j   -30.575216  -26.712822j
 -1130.1614  -1007.9591j     -517.9835   +151.15631j  ]
pos=1603
(2, 3)
Fc is [[[ 18125. -18875.]]]
[-1339.4727-1285.4387j   -1727.1072+3137.7427j    4993.9185-1838.2483j
 -1762.723  -104.444176j]
pos=1604
(2, 4)
Fc is [[[ 18125. -18875.]]]
[  179.712 -3053.9436j -1449.998 +2907.753j  -1876.6797-1396.2778j
 -1307.2443+4956.738j ]
pos=1604
(2, 5)
Fc is [[[ 18125. -18875.]]]
[  318.485 -2081.36j     -1128.1986  -66.844955j   583.9093-4091.6582j
 -2

...

pos=811
(0, 1, 2, 3, 4, 6)
Fc is [[[ 18125. -18875.]]]
[  540.5229  +625.71027j  -931.0271  +559.9182j    968.8     +997.39856j
  -203.10999 +758.5959j   1713.7205  -978.92236j  1194.7869 +1905.4957j
   622.1498  +992.2523j    680.2612  +102.93164j -1054.5132  +805.05237j
  -108.34511 +573.2443j   -996.5845   -39.02762j   785.1753  -346.72592j]
pos=811
(0, 1, 2, 3, 5, 6)
Fc is [[[ 18125. -18875.]]]
[  583.97186 +625.9846j  -1182.568   +630.28937j   560.14386+1078.3574j
  -179.88623+1081.2262j   1090.6823  -928.10693j  1142.3912 +1872.772j
   374.151  +1134.7942j    584.6599  -187.23465j   262.3548  +441.87476j
   -41.65634+1084.721j   -1053.8605   +62.1651j    588.1306  -389.5573j ]
pos=811
(0, 1, 2, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[  437.33722 +665.5316j   -980.86646 +363.48233j  1030.8623 +1035.9298j
  -181.13687 +886.7497j   1526.1119  -760.8659j   1107.346  +1816.786j
 -1413.8843 +1073.888j      72.56592 +169.07019j  1125.9847  +414.38675j
   420.22534+1374.4193j  -1040.2345  

pos=849
(0, 1, 2, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[   4314.7153  +8076.283j    -7559.147   -9699.127j
 -119899.766  -44133.23j      7818.502  -26953.17j
   -8454.498   -4804.114j     -186.53691 +9132.035j
    3199.7834  +1106.1815j   -3748.053  +15197.933j
    7021.4097  -2122.7039j     712.70245 -1964.8887j
    -666.0976  +5245.207j     4518.5283  -3727.231j ]
pos=849
(0, 1, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[   2961.339   +8690.604j    -7594.0874  -9560.054j
 -114009.7    -38207.93j      6261.9453 -28570.389j
     571.4968  +3446.082j    -5345.953   +8807.306j
    3374.2021  -2517.1304j   -1647.0698 +12608.904j
    4170.344   -4404.837j     4261.7715   -916.6094j
    -135.50671 +4595.576j     3836.0686  -3532.1362j]
pos=849
(0, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[ 3431.0469   +8684.431j   -7008.68     -9745.158j
 -8910.951    -6209.5254j   3084.7468   +6367.6064j
  1129.7045   +5314.351j   -6146.1772   +8292.796j
  2130.2534    -285.16614j -2846.4966  +12824.45j
  

pos=781
(0, 1, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[  3241.2876   -893.6139j  15443.025   +1574.6123j
 169076.95   -34916.152j  128558.03  +124246.734j
   5422.361   +6897.0273j -16679.111   -1647.2806j
  -3938.685   -9589.473j  -11030.882   -7703.9155j
   1922.8567  -1991.35j    -3988.1013  +2114.8223j
   6494.4326  -1397.666j    2471.9321  +3084.7505j]
pos=781
(0, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[  6898.0234  -3917.0576j  15120.983   +1307.8472j
    220.51196 +4264.2827j  -9579.63   +10440.605j
   4211.295   +7810.173j  -16752.945   -4155.5005j
  -6567.6123  -7914.898j  -10509.4375  -7732.3423j
   2851.3613  -3323.754j     241.59326 -2905.3013j
   5890.332   -1031.1249j   2983.5107  +2433.6099j]
pos=781
(1, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[151469.95   -44738.273j  138168.14  +122761.016j
   2368.7446  +2590.341j   -7523.629  +10861.545j
   4656.7783  +7259.4443j -13543.312   -2656.j
  -8612.785   -7684.6465j  -6416.7754  -7007.679j
   4093.1304  -4799.045j  

pos=778
(0, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[   88.06092 +211.58566j   383.67288 -376.3905j   -960.8067 -2417.2065j
 -3054.4915 +3523.1j       181.62878+1593.4109j   5168.406  +1591.4479j
 -1804.9564 +2279.2046j    210.22882+4609.158j    6193.2715 +5015.649j
 -1972.3992 -4056.9707j  -2418.7947 -1969.6492j   -935.84485+1786.5396j ]
pos=778
(1, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[   91.58482   -18.77452j     17.621881  +32.711742j
 -1273.5762  -2271.5298j   -3315.212   +3418.6091j
  -276.49158 +1325.4945j    5639.293    +812.8326j
 -2168.435   +2197.6663j     264.55548 +4614.407j
  6485.8555  +4729.0566j   -1702.0397  -3900.0205j
 -2458.0479  -1809.3236j    -781.5194  +1650.258j   ]
pos=778
Trial: 19
Num_Hyd: 6
(0, 1, 2, 3, 4, 5)
Fc is [[[ 18125. -18875.]]]
[  111.99869 +9.5747978e+01j  -106.706955+1.0869160e+01j
   -34.443523-6.7291153e+01j   -14.585165+1.7637300e+00j
 -1531.2424  +7.4573419e+02j -2470.6611  -8.6095068e+02j
    71.86075 -2.4007085e+03j -1270.2814  +7.

pos=793
Trial: 18
Num_Hyd: 7
(0, 1, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[   99.03858  +210.74562j    392.6227   -383.73544j
    94.3661    -19.224861j    17.459534  +34.87967j
 -1010.1351  -2676.4155j   -3074.934   +3618.5225j
   -76.95557 +1572.8046j    5253.6753  +1263.6732j
 -1815.9325  +2228.7683j     330.3709  +4794.645j
  6461.001   +4918.181j    -1878.1594  -4095.45j
 -2385.2544  -2014.5752j    -913.9601  +1685.6863j  ]
pos=778
Trial: 19
Num_Hyd: 7
(0, 1, 2, 3, 4, 5, 6)
Fc is [[[ 18125. -18875.]]]
[  111.49126 +9.6069778e+01j   -98.78806 +1.3374607e+01j
   -34.442276-6.6002869e+01j   -13.920675+5.2996480e-01j
 -1638.3816  +1.0739700e+03j -3190.5808  -3.4341241e+02j
  -462.44598 -1.9697065e+03j -2096.812   +2.7868944e+01j
   582.0216  -3.2593296e+03j   213.2743  +2.5556987e+03j
  -578.21216 -2.2058960e+01j -1118.8331  -1.4071750e+03j
  1253.6466  +3.0984543e+03j  1813.5853  -3.9248176e+03j]
pos=797
Could not find trial 20, skipping...
Could not find trial 21, skipping...
C

In [ ]:
snr_dis = np.zeros((10,7))
ber_dis = np.zeros((10,7))

snr_dis.astype("float32").tofile(f"matplot/med_snr_multihyd_vs_range_single_foam_{data_rate}bps_{volt}Vrms_4x2va_{datestr}.csv",sep=",")
ber_dis.astype("float32").tofile(f"matplot/med_ber_multihyd_vs_range_single_foam_{data_rate}bps_{volt}Vrms_4x2va_{datestr}.csv",sep=",")

In [ ]:
snr_t = np.fromfile(f"matplot/med_snr_multihyd_vs_range_single_foam_{data_rate}bps_{volt}Vrms_4x2va_{datestr}.csv",dtype="float32",sep=",")
snr_t.size

In [ ]:
x = np.zeros((5,6))
print(np.shape(x))

In [ ]:
len([1,2,3,4,9])

In [ ]:
arr = []
for i in range(0,3):
    arr.append([1, 2, 3, 4])
    
print(arr[i])    

In [ ]:
print(np.shape(arr))

In [ ]:
x = 5
if x == 3:
    print(x)

In [19]:
list(combinations([0,1,2,3,4,5,6],2))
x = np.array([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]])

x[0,(1,4)]

array([2, 5])